<a href="https://colab.research.google.com/github/Rizkiramdani04/Portofolio_DS_DA/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
data=pd.read_csv('/content/Salinan Online Retail Data (3).csv')
data

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5.0,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1.0,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5.0,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1.0,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1.0,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
369364,531900,21107,CREAM SLICE FLANNEL PINK SPOT,1.0,2010-11-10 12:03:00,2.95,15031.0
369365,531900,22423,REGENCY CAKESTAND 3 TIER,2.0,2010-11-10 12:03:00,12.75,15031.0
369366,531900,21982,PACK OF 12 SUKI TISSUES,2.0,2010-11-10 12:03:00,0.29,15031.0
369367,531900,20973,12 PENCIL SMALL TUBE WOODLAND,2.0,2010-11-10 12:03:00,0.65,15031.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369369 entries, 0 to 369368
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      369369 non-null  object 
 1   product_code  369368 non-null  object 
 2   product_name  366849 non-null  object 
 3   quantity      369368 non-null  float64
 4   order_date    369368 non-null  object 
 5   price         369368 non-null  float64
 6   customer_id   296074 non-null  float64
dtypes: float64(3), object(4)
memory usage: 19.7+ MB


In [ ]:
df_clean = data.copy()
# membuat kolom date
df_clean['date'] = pd.to_datetime(df_clean['order_date'])
# menghapus semua baris tanpa customer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]
# mengkonversi customer_id menjadi string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
# menghapus semua baris tanpa product_name
df_clean = df_clean[~df_clean['product_name'].isna()]
# membuat semua product_name berhuruf kecil
df_clean['product_name'] = df_clean['product_name'].str.lower()
# menghapus semua baris dengan product_code atau product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
# menghapus baris dengan status cancelled, yaitu yang order_id-nya diawali 'C'
df_clean = df_clean[df_clean['order_id'].str[:1]!='C']
# mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled
df_clean['quantity'] = df_clean['quantity'].abs()
# menghapus baris dengan price bernilai negatif
df_clean = df_clean[df_clean['price']>0]
# membuat nilai amount, yaitu perkalian antara quantity dan price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# menghapus outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

<ipython-input-4-e9f5b530bf16>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['quantity'] = df_clean['quantity'].abs()


,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,amount
0,493414,21844,red retrospot mug,36.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04 10:28:00,91.80
1,493414,21533,retro spot large milk jug,12.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04 10:28:00,51.00
2,493414,37508,new england ceramic cake server,2.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04 10:28:00,5.10
3,493414,35001G,hand open shape gold,2.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04 10:28:00,8.50
4,493414,21527,red retrospot traditional teapot,12.0,2010-01-04 10:28:00,6.95,14590.0,2010-01-04 10:28:00,83.40
...,...,...,...,...,...,...,...,...,...
287117,531900,22614,pack of 12 spaceboy tissues,2.0,2010-11-10 12:03:00,0.29,15031.0,2010-11-10 12:03:00,0.58
287118,531900,21107,cream slice flannel pink spot,1.0,2010-11-10 12:03:00,2.95,15031.0,2010-11-10 12:03:00,2.95
287119,531900,22423,regency cakestand 3 tier,2.0,2010-11-10 12:03:00,12.75,15031.0,2010-11-10 12:03:00,25.50
287120,531900,21982,pack of 12 suki tissues,2.0,2010-11-10 12:03:00,0.29,15031.0,2010-11-10 12:03:00,0.58


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287122 entries, 0 to 287121
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      287122 non-null  object        
 1   product_code  287122 non-null  object        
 2   product_name  287122 non-null  object        
 3   quantity      287122 non-null  float64       
 4   order_date    287122 non-null  object        
 5   price         287122 non-null  float64       
 6   customer_id   287122 non-null  object        
 7   date          287122 non-null  datetime64[ns]
 8   amount        287122 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 19.7+ MB


In [ ]:
basket=pd.pivot_table(df_clean,index='order_id',columns='product_name',values='product_code',aggfunc='nunique',fill_value=0)
basket

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531875,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
531881,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
531894,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13361 entries, 493414 to 531900
Columns: 3775 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: int64(3775)
memory usage: 384.9+ MB


In [ ]:
def encode(x):
  if x==0:
    return False
  if x>0:
    return True
basket_encode=basket.applymap(encode)
basket_encode

<ipython-input-8-cf62d104603e>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encode=basket.applymap(encode)


product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531875,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
531881,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
531894,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
basket_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13361 entries, 493414 to 531900
Columns: 3775 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3775)
memory usage: 48.2+ MB


In [ ]:
basket_filter=basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531875,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
531881,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
531894,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
basket_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12313 entries, 493414 to 531900
Columns: 3775 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3775)
memory usage: 44.4+ MB


In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemset=apriori(basket_filter,min_support=.01,use_colnames=True).sort_values('support',ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt']=frequent_itemset['itemsets'].apply(lambda x:len(x))
frequent_itemset

,support,itemsets,product_cnt
0,0.182328,(white hanging heart t-light holder),1
1,0.100382,(jumbo bag red retrospot),1
2,0.099488,(regency cakestand 3 tier),1
3,0.084870,(pack of 72 retro spot cake cases),1
4,0.082839,(lunch bag red spotty),1
...,...,...,...
1245,0.010071,"(sweetheart ceramic trinket box, ceramic straw...",2
1246,0.010071,"(regency cakestand 3 tier, pack of 60 pink pai...",2
1247,0.010071,(boys vintage tin seaside bucket),1
1248,0.010071,"(bright blues ribbons, lush greens ribbons)",2


In [ ]:
from mlxtend.frequent_patterns import association_rules
product_association=association_rules(frequent_itemset,metric='confidence',min_threshold=.7).sort_values(['support','confidence'],ascending=[False,False]).reset_index(drop=True)
product_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(red hanging heart t-light holder),(white hanging heart t-light holder),0.055795,0.182328,0.040770,0.730713,4.007694,1.0,0.030597,3.036437,0.794827,0.206584,0.670667,0.477161
1,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.052871,0.080646,0.040364,0.763441,9.466513,1.0,0.036100,3.886358,0.944290,0.433304,0.742690,0.631972
2,(wooden picture frame white finish),(wooden frame antique white),0.046699,0.062454,0.033217,0.711304,11.389194,1.0,0.030300,3.247523,0.956883,0.437433,0.692073,0.621582
3,(toilet metal sign),(bathroom metal sign),0.030537,0.044912,0.024689,0.808511,18.002155,1.0,0.023318,4.987682,0.974200,0.486400,0.799506,0.679120
4,(pink 3 piece mini dots cutlery set),(blue 3 piece mini dots cutlery set),0.030537,0.032405,0.021441,0.702128,21.667413,1.0,0.020451,3.248355,0.983893,0.516634,0.692152,0.681891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,(pink regency teacup and saucer),"(roses regency teacup and saucer, green regenc...",0.012832,0.012182,0.010152,0.791139,64.941983,1.0,0.009996,4.729552,0.997400,0.683060,0.788563,0.812236
133,(green spotty cup),(blue spotty cup),0.013319,0.019898,0.010152,0.762195,38.305749,1.0,0.009887,4.121456,0.987041,0.440141,0.757367,0.636200
134,(set/6 turquoise butterfly t-lights),(set/6 pink butterfly t-lights),0.014050,0.016893,0.010152,0.722543,42.772482,1.0,0.009915,3.543283,0.990538,0.488281,0.717776,0.661752
135,(poppy's playhouse kitchen),(poppy's playhouse livingroom),0.014131,0.010639,0.010152,0.718391,67.523252,1.0,0.010002,3.513241,0.999312,0.694444,0.715363,0.836295
